In [1]:
import scraping_class
import pandas as pd
import requests,os,time
from datetime import datetime

In [2]:
def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(1) # sleep one second.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring.
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessment
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case).
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type

    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile\n",
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
           
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
  
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
 
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\\n'+';'.join(map(str,row))) # write log.
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\\n'+';'.join(map(str,row))) # write row to log.
    else:
      t = time.time()
      ratelimit()
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call.
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\\n'+';'.join(map(str,row))) # write row to log file.
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return call_id

logfile="tripadvisor_loader.txt" # name your log file.
connector = Connector(logfile)

In [3]:
url_init = 'https://www.tripadvisor.com' # base url
max_restaurant = 2322
init_number = 0
final_urls = []
timestamp = datetime.now()
year = timestamp.year
month = timestamp.month
day = timestamp.day

while init_number <= max_restaurant:
    timestamp = datetime.now() # Creates a timestamp in the format yyyy-mm-dd h:m:s
    
    url_search = '/RestaurantSearch-g189541-o' # first part of url
    url_search2 = '-a_date.2019__2D__08__2D__22-a_people.2-a_time.20%3A00%3A00-a_zur.2019__5F__08__5F__22-Copenhag.html#EATERY_LIST_CONTENTS' # second part of url
    url_search3 = url_init + url_search + str(init_number) + url_search2 # combined dynamic url
    data, call_id = connector.get(url_search3, 'exam_init') # gathering data
    response = data.ok # checks if datastream is possible
    final_urls.append(url_search3)

    print('Status:', response, 'Timestamp:', timestamp, 'links processed:', len(final_urls)) # prints response, timestamp and amount of links processed.
    init_number += 30 # incriments the number by 30 to be put into the url on next call."

Status: True Timestamp: 2019-08-23 13:34:40.367359 links processed: 1
Status: True Timestamp: 2019-08-23 13:34:42.674099 links processed: 2
Status: True Timestamp: 2019-08-23 13:34:45.182122 links processed: 3
Status: True Timestamp: 2019-08-23 13:34:47.111715 links processed: 4
Status: True Timestamp: 2019-08-23 13:34:49.065994 links processed: 5
Status: True Timestamp: 2019-08-23 13:34:51.226448 links processed: 6
Status: True Timestamp: 2019-08-23 13:34:53.194715 links processed: 7
Status: True Timestamp: 2019-08-23 13:34:55.905950 links processed: 8
Status: True Timestamp: 2019-08-23 13:34:58.371259 links processed: 9
Status: True Timestamp: 2019-08-23 13:35:00.384629 links processed: 10
Status: True Timestamp: 2019-08-23 13:35:03.078650 links processed: 11
Status: True Timestamp: 2019-08-23 13:35:05.597798 links processed: 12
Status: True Timestamp: 2019-08-23 13:35:08.040681 links processed: 13
Status: True Timestamp: 2019-08-23 13:35:09.979516 links processed: 14
Status: True Ti

In [4]:
print(init_number)
print(len(final_urls))
overview_urls = pd.DataFrame(final_urls)
pd.options.display.max_colwidth = 200
print(overview_urls.head())

2340
78
                                                                                                                                                                                    0
0    https://www.tripadvisor.com/RestaurantSearch-g189541-o0-a_date.2019__2D__08__2D__22-a_people.2-a_time.20%3A00%3A00-a_zur.2019__5F__08__5F__22-Copenhag.html#EATERY_LIST_CONTENTS
1   https://www.tripadvisor.com/RestaurantSearch-g189541-o30-a_date.2019__2D__08__2D__22-a_people.2-a_time.20%3A00%3A00-a_zur.2019__5F__08__5F__22-Copenhag.html#EATERY_LIST_CONTENTS
2   https://www.tripadvisor.com/RestaurantSearch-g189541-o60-a_date.2019__2D__08__2D__22-a_people.2-a_time.20%3A00%3A00-a_zur.2019__5F__08__5F__22-Copenhag.html#EATERY_LIST_CONTENTS
3   https://www.tripadvisor.com/RestaurantSearch-g189541-o90-a_date.2019__2D__08__2D__22-a_people.2-a_time.20%3A00%3A00-a_zur.2019__5F__08__5F__22-Copenhag.html#EATERY_LIST_CONTENTS
4  https://www.tripadvisor.com/RestaurantSearch-g189541-o120-a_date.2019__2D__08__

In [5]:
overview_urls.columns = ['Init_links']
overview_urls.to_csv('overview_urls.csv', index = None, header = True)

In [6]:
page_list = []

for url in final_urls: # loops over all overview sites
    response,call_id = connector.get(url,'course_mapping')
    link_locations = response.text.split('href="')[1:] # find all links on site 
    link_list = [] 
    
    for i in link_locations:
        if "Restaurant_Review" in i: # finds all links named something with Restaurant review
            link_list.append(i.partition(" ")[0])

        review_list=  []
    for i in link_list:
        if "Restaurant_Review" in i and "#REVIEWS" not in i and "button" not in i: # same link appars more times this loop removes two alternative versions
            review_list.append(i)

            review_list_u = set(review_list) # remove dublicates

            url_part_1 = "https://www.tripadvisor.com/"  # this is the baseline part of a link

    for i in review_list_u:
        timestamp = datetime.now()
        page_list.append(url_part_1+i) # make list of final urls
        print('Timestamp:', timestamp, 'Links processed:', len(page_list)) # prints timestamp and amount of links processed.       

Timestamp: 2019-08-23 13:37:28.669442 Links processed: 1
Timestamp: 2019-08-23 13:37:28.669618 Links processed: 2
Timestamp: 2019-08-23 13:37:28.669703 Links processed: 3
Timestamp: 2019-08-23 13:37:28.669784 Links processed: 4
Timestamp: 2019-08-23 13:37:28.669869 Links processed: 5
Timestamp: 2019-08-23 13:37:28.669950 Links processed: 6
Timestamp: 2019-08-23 13:37:28.670029 Links processed: 7
Timestamp: 2019-08-23 13:37:28.670114 Links processed: 8
Timestamp: 2019-08-23 13:37:28.670195 Links processed: 9
Timestamp: 2019-08-23 13:37:28.670280 Links processed: 10
Timestamp: 2019-08-23 13:37:28.670361 Links processed: 11
Timestamp: 2019-08-23 13:37:28.670446 Links processed: 12
Timestamp: 2019-08-23 13:37:28.670526 Links processed: 13
Timestamp: 2019-08-23 13:37:28.670610 Links processed: 14
Timestamp: 2019-08-23 13:37:28.670690 Links processed: 15
Timestamp: 2019-08-23 13:37:28.670773 Links processed: 16
Timestamp: 2019-08-23 13:37:28.677938 Links processed: 17
Timestamp: 2019-08-23 1

In [7]:
print(len(page_list))
indvidual_urls = pd.DataFrame(page_list)

2319


In [8]:
indvidual_urls.columns = ['Restaurant_links']
indvidual_urls.to_csv('indvidual_urls.csv', index = None, header = True)